"""
AI Legal Advisor - モデル推論テスト（Python スクリプト版）

このスクリプトは Jupyter Notebook と同じ内容を実行します。
Google Colab で実行する場合は、.ipynb 版を使用してください。
"""

# ==========================================
# 1. 環境セットアップ
# ==========================================

# Google Driveをマウント（Colabのみ）
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IS_COLAB = True
except ImportError:
    IS_COLAB = False
    print("⚠️ Google Colab環境ではありません")

# 必要なライブラリ
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import time
import json
from typing import Dict, Any

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ==========================================
# 2. モデルのロード
# ==========================================

# モデルのパス（実際のパスに変更してください）
if IS_COLAB:
    MODEL_PATH = "/content/drive/MyDrive/your-model-path/"
else:
    MODEL_PATH = "./model/"  # ローカルの場合

# Elyza-7Bのベースモデル名
BASE_MODEL_NAME = "elyza/ELYZA-japanese-Llama-2-7b-instruct"

print(f"Loading model from: {MODEL_PATH}")

# 4bit量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# トークナイザーのロード
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

# モデルのロード
print("Loading model... (This may take a few minutes)")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

print("✅ Model loaded successfully!")

# ==========================================
# 3. 推論関数の定義
# ==========================================

def generate_legal_advice(
    input_text: str,
    max_new_tokens: int = 450,
    temperature: float = 0.7,
    top_p: float = 0.9,
    top_k: int = 50,
) -> Dict[str, Any]:
    """
    IT法務に関する判定を実行
    
    Args:
        input_text: ユーザーの入力（チェックしたい仕様）
        max_new_tokens: 生成する最大トークン数
        temperature: サンプリング温度
        top_p: nucleus sampling parameter
        top_k: top-k sampling parameter
    
    Returns:
        Dict containing:
            - output: 生成されたテキスト
            - inference_time: 推論時間（秒）
            - tokens_generated: 生成されたトークン数
    """
    # プロンプトテンプレート
    prompt = f"""以下のIT関連の仕様について、法的リスクを判定してください。

仕様:
{input_text}

以下の観点で分析してください:
1. リスクレベル（高/中/低）
2. 該当する可能性のある法律
3. リスクの理由
4. 推奨される対応策

回答:"""
    
    # トークン化
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 推論実行（時間測定）
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    inference_time = time.time() - start_time
    
    # デコード
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # プロンプト部分を除去して回答のみ抽出
    answer = generated_text.split("回答:")[-1].strip()
    
    return {
        "output": answer,
        "inference_time": inference_time,
        "tokens_generated": len(outputs[0]) - len(inputs["input_ids"][0]),
        "full_output": generated_text,
    }

# ==========================================
# 4. テスト実行
# ==========================================

print("\n" + "=" * 80)
print("テストケース実行開始")
print("=" * 80 + "\n")

# テストケース1: 個人情報保護
test_case_1 = "ユーザーの位置情報を収集して、第三者の広告配信事業者に提供します。"

print("=" * 80)
print("テストケース1: 個人情報保護")
print("=" * 80)
print(f"入力: {test_case_1}\n")

result_1 = generate_legal_advice(test_case_1)

print("【判定結果】")
print(result_1["output"])
print("\n【パフォーマンス】")
print(f"推論時間: {result_1['inference_time']:.2f}秒")
print(f"生成トークン数: {result_1['tokens_generated']}")
print("\n")

# テストケース2: 消費者保護（ダークパターン）
test_case_2 = "解約ボタンを画面の一番下に小さく配置し、その上に『本当に解約しますか？多くの特典を失います』という警告を3回表示します。"

print("=" * 80)
print("テストケース2: 消費者保護")
print("=" * 80)
print(f"入力: {test_case_2}\n")

result_2 = generate_legal_advice(test_case_2)

print("【判定結果】")
print(result_2["output"])
print("\n【パフォーマンス】")
print(f"推論時間: {result_2['inference_time']:.2f}秒")
print(f"生成トークン数: {result_2['tokens_generated']}")
print("\n")

# テストケース3: アクセシビリティ
test_case_3 = "重要な操作ボタンを画像のみで表示し、代替テキストを設定していません。"

print("=" * 80)
print("テストケース3: アクセシビリティ")
print("=" * 80)
print(f"入力: {test_case_3}\n")

result_3 = generate_legal_advice(test_case_3)

print("【判定結果】")
print(result_3["output"])
print("\n【パフォーマンス】")
print(f"推論時間: {result_3['inference_time']:.2f}秒")
print(f"生成トークン数: {result_3['tokens_generated']}")
print("\n")

# ==========================================
# 5. パフォーマンス分析
# ==========================================

print("=" * 80)
print("パフォーマンス分析")
print("=" * 80 + "\n")

# 複数回実行して平均を取る
num_runs = 5
test_input = "ユーザーのメールアドレスを同意なく第三者に提供します。"

times = []
print(f"パフォーマンステスト（{num_runs}回実行）...")

for i in range(num_runs):
    result = generate_legal_advice(test_input, max_new_tokens=300)
    times.append(result["inference_time"])
    print(f"Run {i+1}: {result['inference_time']:.2f}秒")

avg_time = sum(times) / len(times)
print(f"\n平均推論時間: {avg_time:.2f}秒")
print(f"最小時間: {min(times):.2f}秒")
print(f"最大時間: {max(times):.2f}秒")
print("\n")

# ==========================================
# 6. 結果の保存
# ==========================================

# テスト結果をJSONとして保存
test_results = {
    "test_case_1": {
        "input": test_case_1,
        "output": result_1["output"],
        "inference_time": result_1["inference_time"],
    },
    "test_case_2": {
        "input": test_case_2,
        "output": result_2["output"],
        "inference_time": result_2["inference_time"],
    },
    "test_case_3": {
        "input": test_case_3,
        "output": result_3["output"],
        "inference_time": result_3["inference_time"],
    },
    "performance": {
        "average_time": avg_time,
        "min_time": min(times),
        "max_time": max(times),
    }
}

# 保存先の決定
if IS_COLAB:
    output_path = '/content/drive/MyDrive/model_test_results.json'
else:
    output_path = './model_test_results.json'

# JSONファイルとして保存
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(test_results, f, ensure_ascii=False, indent=2)

print(f"✅ テスト結果を保存しました: {output_path}")

# ==========================================
# 7. サマリー
# ==========================================

print("\n" + "=" * 80)
print("テスト完了サマリー")
print("=" * 80)
print(f"✅ テストケース実行: 3件")
print(f"✅ パフォーマンステスト: {num_runs}回")
print(f"✅ 平均推論時間: {avg_time:.2f}秒")
print(f"✅ 結果保存先: {output_path}")
print("\n次のステップ:")
print("1. FastAPIでこの推論機能をAPI化")
print("2. ngrokで外部公開")
print("3. Streamlitアプリから呼び出し")
print("=" * 80)